In [1]:
!pip install --upgrade keras
!pip install -U keras-nlp
!pip install keras-cv
!apt-get install wget 


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
wget is already the newest version (1.21.2-2ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 43 not upgraded.


In [2]:
IMG_HEIGHT = 240
IMG_WIDTH = 240
BATCH_SIZE = 3
EPOCHS = 2000
checkpoint_filepath = "weights-improvement-{epoch:02d}-{loss:.2f}.keras"

In [3]:
from keras import ops
from keras_cv.models.feature_extractor import CLIPImageEncoder
from keras.callbacks import ModelCheckpoint
from keras_cv.models import CLIP
from keras.layers import Input,Concatenate, Layer,Dense,Reshape,MultiHeadAttention,LayerNormalization,Add,Flatten
from keras import Sequential,Model
from keras.optimizers import Adam
import tensorflow as tf
import os
os.environ["KERAS_BACKEND"] = "tensorflow"

2024-11-28 22:01:04.954884: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732831264.969243     669 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732831264.973263     669 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-28 22:01:04.988948: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
model = CLIP.from_preset("clip-vit-base-patch32")
image_encoder = model.get_layer("image_encoder")
image_encoder.trainable = False

W0000 00:00:1732831267.301752     669 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [5]:
class AttentionDecoder(Layer):
    def __init__(self, num_heads, embed_dim, ff_dim):
        super().__init__()
        self.attention = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.norm1 = LayerNormalization(epsilon=1e-6)
        self.add1 = Add()
        
        self.ffn = Sequential([
            Dense(ff_dim, activation="gelu"),
            Dense(embed_dim),
        ])
        self.norm2 = LayerNormalization(epsilon=1e-6)
        self.add2 = Add()

    def call(self, inputs):
        attn_output = self.attention(inputs, inputs)
        out1 = self.add1([inputs, attn_output])
        out1 = self.norm1(out1)
        ffn_output = self.ffn(out1)
        out2 = self.add2([out1, ffn_output])
        return self.norm2(out2)


In [6]:
object_input = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
background_input =  Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
encoded_image_1 = image_encoder(object_input)
encoded_image_2 = image_encoder(background_input)
concatenated_inputs  = Concatenate()([encoded_image_1,encoded_image_2])
reshaped_features = Reshape((16, 64))(concatenated_inputs)
decoder = AttentionDecoder(num_heads=8, embed_dim=64, ff_dim=1024)
x = decoder(reshaped_features)
x = decoder(x)
x = decoder(x)
x = decoder(x)
x = decoder(x)
x = decoder(x)
x = decoder(x)
x = decoder(x)
x = decoder(x)
x = decoder(x)
x = decoder(x)
x = decoder(x)
x = Flatten()(x)
x = Dense(64, activation="gelu")(x)
x = Dense(32, activation="gelu")(x)
x = Dense(240 * 240 * 3)(x)
output_image =  Reshape((240, 240, 3))(x)
model = Model(inputs=[object_input, background_input], outputs=output_image)
checkpoint = ModelCheckpoint(checkpoint_filepath,monitor='loss',verbose=1,save_best_only=True)
callbacks_list = [checkpoint]
model.compile(optimizer=Adam(learning_rate=1e-4), loss="mse")
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 240, 240,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 240, 240,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ image_encoder       │ (None, 512)       │ 87,849,216 │ input_layer[0][0… │
│ (CLIPImageEncoder)  │                   │            │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 1024)      │          0 │ image_encoder[1]… │
│ (Concatenate)       │                   │            │ image_encoder[2]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 16, 64)    │          0 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_decoder   │ (None, 16, 64)    │    265,088 │ reshape[0][0],    │
│ (AttentionDecoder)  │                   │            │ attention_decode… │
│                     │                   │            │ attention_decode… │
│                     │                   │            │ attention_decode… │
│                     │                   │            │ attention_decode… │
│                     │                   │            │ attention_decode… │
│                     │                   │            │ attention_decode… │
│                     │                   │            │ attention_decode… │
│                     │                   │            │ attention_decode… │
│                     │                   │            │ attention_decode… │
│                     │                   │            │ attention_decode… │
│                     │                   │            │ attention_decode… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 1024)      │          0 │ attention_decode… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 64)        │     65,600 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 32)        │      2,080 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 172800)    │  5,702,400 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_1 (Reshape) │ (None, 240, 240,  │          0 │ dense_4[0][0]     │
│                     │ 3)                │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 93,884,384 (358.14 MB)

 Trainable params: 6,035,168 (23.02 MB)

 Non-trainable params: 87,849,216 (335.12 MB)

In [7]:
import pickle

data = None
with open('images_dict.pkl', 'rb') as file:
    data = pickle.load(file)

In [8]:
object_dir = '/tf/notebooks/targets'
background_dir = '/tf/notebooks/backgrounds'
original_image_dir = '/tf/notebooks/awsaf49/coco-2017-dataset/versions/2/coco2017/train2017'

In [9]:
def load_and_preprocess_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_image(img, channels=3)
    img.set_shape([None, None, 3])
    img = tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])
    img = tf.cast(img, tf.float32) / 255.0
    return img

In [10]:
object_paths = list()
background_paths = list()
original_image_paths = list()

In [11]:
for key,value in data.items():
    if len(value["targets"]) == len(value["backgrounds"]):
        original_image_path = original_image_dir+"/" + key +".jpg"
        for object_path,bg_path in zip(value["targets"],value["backgrounds"]):
            if isinstance(object_path, str) and isinstance(bg_path, str) and isinstance(original_image_path,str):
                
                object_paths.append(os.getcwd()+"/"+object_path)
                background_paths.append(os.getcwd()+"/"+bg_path)
                original_image_paths.append(original_image_path)

In [12]:
if not (len(object_paths) == len(background_paths) == len(original_image_paths)):
    raise ValueError("All directories must have the same number of images.")

In [13]:
object_dataset = tf.data.Dataset.from_tensor_slices(object_paths).map(load_and_preprocess_image)

In [14]:
background_dataset = tf.data.Dataset.from_tensor_slices(background_paths).map(load_and_preprocess_image)
original_image_dataset = tf.data.Dataset.from_tensor_slices(original_image_paths).map(load_and_preprocess_image)

In [15]:
dataset = tf.data.Dataset.zip(((object_dataset, background_dataset), original_image_dataset)).batch(BATCH_SIZE,drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)

In [16]:
model.fit(dataset,epochs=EPOCHS,callbacks=callbacks_list,verbose=1)

Epoch 1/2000
256/256 ━━━━━━━━━━━━━━━━━━━━ 0s 345ms/step - loss: 0.1336
Epoch 1: loss improved from inf to 0.09639, saving model to weights-improvement-01-0.10.keras
256/256 ━━━━━━━━━━━━━━━━━━━━ 107s 355ms/step - loss: 0.1334
Epoch 2/2000
254/256 ━━━━━━━━━━━━━━━━━━━━ 0s 350ms/step - loss: 0.0784

KeyboardInterrupt: 